In [651]:
# Nem todos os trimestres estão sendo incluídos no dataset final.
# A fonte dos dados não é muito boa, existem balanços apenas com versões posteriores à primeira.
# Além disso, muitos relatórios do último trimestre não estão sendo incluídos pois eles, na verdade, são inseridos 
# nos documentos do primeiro trimestre do ano seguinte com o 'ORDEM_EXERC' como 'PENÚLTIMO'.

# Reduzir o número de empresas abordadas. Usar, por ora, umas 3 ou 5 (e das grandes) -ITAU, PETR3 e AMBEV.
# Sofisticar o método de coleta dos dados!

<a href='https://www.idinheiro.com.br/cnpj-empresas-listadas-b3/'> CNPJ e Ticker </a>

<h1 style='font-size:40px'> Stock Price Variation Predictor</h1>
<div> 
    <ul style='font-size:20px'> 
        <li> 
            The present project aims to create a stock price variation predictor using the  BOVESPA corporations' fundamentals. 
        </li>
        <li> 
            The model must estimate the given quarter's 80th percentile variation using the financial statements from the previous quarter.
        </li>
    </ul>
</div>

<h2 style='font-size:30px'> Building the dataset</h2>
<div> 
    <ul style='font-size:20px'> 
        <li> 
            Unlike most of the ML projects out there, we'll face the challenge of making the dataset with which our models will carry out their predictions.
        </li>
        <li> 
            We are going to use wget in order to download the files and extract the financial statements. The final dataset will store data from 2015 to 2021. 
        </li>
    </ul>
</div>

<a href='https://www.youtube.com/watch?v=LCMgVO1BOVQ&t=659s'> Link Tutorial</a>

In [84]:
import wget
import pandas as pd
from datetime import date
from zipfile import ZipFile
from pathlib import Path
import os
import numpy as np

In [9]:
import wget
import pandas as pd
from datetime import date
from zipfile import ZipFile
from pathlib import Path
import os

# The data source to the financial statements is going to be the below URL.
data_source = 'http://dados.cvm.gov.br/dados/CIA_ABERTA/DOC/ITR/DADOS/'

# Listing the documents from 2015 to last year.
# It is worth considering that the actual statements are stored in zip files. So we'll need to use some method to extract them.
current_year = date.today().year
zip_files = [f'itr_cia_aberta_{year}.zip' for year in range(2015, current_year)]

# Downloading the files and extracting their contents.
for file in zip_files:
    wget.download(data_source + file)
    with ZipFile(file, 'r') as zip_file:
        zip_file.extractall('Statements')
    # After the extraction is completed, we are able to delete the zip file.
    os.remove(file)

# The zip files contained two kinds of reports, the ones with '_con_' and others with '_ind_' in their names.
# Since we are going to use only the first sort of document, we'll remove the '_ind_' ones.

# Also, there were two kinds of DMPL statements, the '_MD_' and the '_MI_'. I've chosen to keep with the second type.
for file in Path('Statements').iterdir():
    if '_ind_' in file.name or '_MI_' in file.name:
        os.remove(file)

100% [....................................................] 29984400 / 29984400

In [47]:
with open('.gitignore', 'w') as git:
    for file in os.listdir('Statements'):
        git.write(f'Statements/{file}\n')
            
git.close()

In [652]:
from re import search, findall
# This function creates a DataFrame with all the reports of a certain kind of statement.
def unite_reports(type_statement):
    df = pd.DataFrame()
    for file in os.listdir('Statements'):
        if search(type_statement, file):
            financial_statement_df = pd.read_csv(f'Statements/{file}', encoding='ISO-8859-1', sep=';', parse_dates=['DT_FIM_EXERC'],
                                                na_values=0)
            
            # Filtering out some rows that are not of our interest.
            financial_statement_df = financial_statement_df.query('''ORDEM_EXERC=='ÚLTIMO' & ST_CONTA_FIXA=='S' & VERSAO==1 ''')
            df = pd.concat([df, financial_statement_df])

    pivot = pd.pivot_table(df, values='VL_CONTA', columns='CD_CONTA', index=['CNPJ_CIA', 'DT_FIM_EXERC'])
    # Retrieving a DataFrame with the accounts' percentage change. Removing the lines that based the pct change comparison (since they
    # are all NaN).
    return pivot#.groupby('CNPJ_CIA').pct_change().dropna(how='all').fillna(method='bfill')

# This method converts the DF's level 1 data type to PeriodIndex.
def period_index(df):
    quarter_index = pd.PeriodIndex(df.xs(key=slice(None), level=0).index, freq='Q')
    return df.set_index([df.xs(key=slice(None), level=1).index , quarter_index])

# The 'accounts' method puts all the companies' accounts alligned per quarter.
def accounts():
    df = unite_reports('BPA')
    for report in ['BPP', 'DFC', 'DMPL', 'DRA', 'DRE', 'DVA']:
        df = df.join(unite_reports(report))
    # I've decided to filter out corporations with less than 12 quarters of available data. They may be unreliable to analysis.
    sizes = df.groupby('CNPJ_CIA').size()
    df.loc(axis=0)[sizes[sizes>12].index,:]
    
    return period_index(df)

# Adding the stocks' tickers using the CNPJ from the companies.
def add_tickers():
    df = accounts()
    # I've found on the internet a dataset that shows us the corporations' CNPJ along their correspondent BOVESPA ticker.
    acoes=pd.read_csv('CNPJ-Ticker.csv', index_col=1, usecols=[1,2])
    acoes.rename({'Código(s)':'Ticker'}, axis=1, inplace=True)
    
    # Some of the companies hold more than one ticker, so we are going to choose a single of them to evaluate.
    for i, ticker in zip(acoes.index,acoes['Ticker']):
        if len(ticker)>6:
            acoes.loc[i, 'Ticker'] = findall('[A-Z]{4}3',acoes.loc[i, 'Ticker'])[0]
            
    # Returning the dataset with the stock tickers.
    return pd.merge(df.reset_index(), acoes, left_on='CNPJ_CIA', right_on='CNPJ').drop('CNPJ_CIA', axis=1).set_index(['Ticker', 'DT_FIM_EXERC'])

# Finally, the 'target_value' function adds the target value we aim to estimate: the 80th percentile from following quarters'
# the closing price.
# In order to provide some context to our algorithms,  the 80th percentile from the current quarter will also be added along with the 
# stock's average volume.
def target_value():
    df = add_tickers()
    # The values to be attached in the DataFrame will firstly be stored in Series.
    index = pd.MultiIndex.from_product([[np.nan], [np.nan]], names=('Ticker', 'DT_FIM_EXERC'))
    prices = pd.Series(dtype='float64', index=index)
    prices_shift = pd.Series(dtype='float64', index=index)
    for ticker in df.index.levels[0]:
            try:
                # Consulting the Yahoo Finance API.
                ticker_df = web.DataReader(f'{ticker}.SA', 'yahoo', '01-01-2015', '12-31-2021')
                
                # Making some small adjustments in 'ticker_df'`s index.
                ticker_df.index.name = 'DT_FIM_EXERC'
                ticker_df.index = pd.PeriodIndex(ticker_df.index, freq='Q')
                
                # Finding the quarters' 80 th closing price percentile.
                quantile_80 = ticker_df.resample('Q')['Close'].quantile(0.8).reset_index()
                
                # Creating a new column with the ticker name. It is going to be useful for the 'attachment' in the dataset.
                quantile_80['Ticker']=ticker
                
                # **O provável erro ocorre aqui ** Testar!!
                quantile_80 = quantile_80.groupby(['Ticker', 'DT_FIM_EXERC'])['Close'].mean()
                
                # Appending the values in the Series generated.
                prices = prices.append(quantile_80)
                prices_shift = prices_shift.append(quantile_80.shift(-1))
            # If no information is retrieved by Yahoo, all the rows regarding the ticker in analysis are removed.
            except:
                df.drop(ticker, axis=0, level=0, inplace=True)
                
    # When we've got all the values, we are going to join the Series to the dataset as new columns.
    prices.name = 'Current Quarter 80th pct'
    prices_shift.name='Following Quarter 80th pct'
    return df.join(prices).join(prices_shift)



In [653]:
df = target_value()
#df = target_value()

In [663]:
df.to_csv('dataset.csv')

In [681]:
# Os valores não batem!
df.loc['ELET3'].head(10).iloc[:, [-2,-1]]

,Current Quarter 80th pct,Following Quarter 80th pct
DT_FIM_EXERC,,
2015Q1,5.577880,7.297227
2015Q2,7.297227,5.795797
2015Q3,5.795797,5.987724
2016Q1,6.795417,8.862632
2016Q2,8.862632,20.694136
2016Q3,20.694136,23.690996
2017Q1,22.327516,16.501730
2017Q3,20.006397,21.227933
2018Q1,23.641016,19.042764


In [685]:
df.head()

1        1.01    1.01.01    1.01.02  \
Ticker DT_FIM_EXERC                                                  
ELET3  2015Q1        152427122.0  34450167.0  2879958.0  4648826.0   
       2015Q2        150896328.0  31126437.0  2152413.0  7011131.0   
       2015Q3        152671980.0  30441004.0  2962477.0  6922589.0   
       2016Q1        149025438.0  28637497.0  3176163.0  5438598.0   
       2016Q2        173048005.0  27273549.0  2716600.0  6239175.0   

                     1.01.02.01  1.01.02.01.01  1.01.02.01.02  1.01.02.02  \
Ticker DT_FIM_EXERC                                                         
ELET3  2015Q1               NaN            NaN            NaN   4648826.0   
       2015Q2               NaN            NaN            NaN   7011131.0   
       2015Q3               NaN            NaN            NaN   6922589.0   
       2016Q1               NaN            NaN            NaN   5438598.0   
       2016Q2               NaN            NaN            NaN   6239175.0   

                     1.01.02.02.01  1.01.02.03    1.01.03  1.01.03.01  \
Ticker DT_FIM_EXERC                                                     
ELET3  2015Q1            4648826.0         NaN  4745292.0   4745292.0   
       2015Q2            7011131.0         NaN  4527083.0   4527083.0   
       2015Q3            6922589.0         NaN  5127948.0   5127948.0   
       2016Q1            5438598.0         NaN  4153132.0   4153132.0   
       2016Q2            6239175.0         NaN  4443974.0   4443974.0   

                     1.01.03.02   1.01.04  1.01.05   1.01.06  1.01.06.01  \
Ticker DT_FIM_EXERC                                                        
ELET3  2015Q1               NaN  873662.0      NaN  537106.0    537106.0   
       2015Q2               NaN  970021.0      NaN  666934.0    666934.0   
       2015Q3               NaN  947745.0      NaN  617931.0    617931.0   
       2016Q1               NaN  992093.0      NaN  499605.0    499605.0   
       2016Q2               NaN  996749.0      NaN  643438.0    643438.0   

                     1.01.07     1.01.08  1.01.08.01  1.01.08.02  1.01.08.03  \
Ticker DT_FIM_EXERC                                                            
ELET3  2015Q1            NaN  20765323.0         NaN         NaN  20765323.0   
       2015Q2            NaN  15798855.0         NaN         NaN  15798855.0   
       2015Q3            NaN  13862314.0         NaN         NaN  13862314.0   
       2016Q1            NaN  14377906.0         NaN         NaN  14377906.0   
       2016Q2            NaN  12233613.0         NaN         NaN  12233613.0   

                     1.01.09  1.01.09.03         1.02     1.02.01  1.02.01.01  \
Ticker DT_FIM_EXERC                                                             
ELET3  2015Q1            NaN         NaN  117976955.0  64189864.0         NaN   
       2015Q2            NaN         NaN  119769891.0  65518317.0         NaN   
       2015Q3            NaN         NaN  122230976.0  67988837.0         NaN   
       2016Q1            NaN         NaN  120387941.0  66400188.0         NaN   
       2016Q2            NaN         NaN  145774456.0  93033264.0         NaN   

                     1.02.01.01.01  1.02.01.01.02  1.02.01.02  1.02.01.02.01  \
Ticker DT_FIM_EXERC                                                            
ELET3  2015Q1                  NaN            NaN    227344.0       227344.0   
       2015Q2                  NaN            NaN    214582.0       214582.0   
       2015Q3                  NaN            NaN    218294.0       218294.0   
       2016Q1                  NaN            NaN    197085.0       197085.0   
       2016Q2                  NaN            NaN    200547.0       200547.0   

                     1.02.01.03  1.02.01.03.01  1.02.01.03.02  1.02.01.04  \
Ticker DT_FIM_EXERC                                                         
ELET3  2015Q1         1776653.0      1776653.0            NaN    623882.0   
       2015Q2         1798617.0   

In [688]:
# Consegui criar o elo entre os resultados da API e o DataFrame!
index = pd.MultiIndex.from_product([[np.nan], [np.nan]], names=('Ticker', 'DT_FIM_EXERC'))
prices = pd.Series(dtype='float64', index=index)
prices_shift = pd.Series(dtype='float64', index=index)
for ticker in ['ELET3']:
        try:
            # Consulting the Yahoo Finance API.
            ticker_df = web.DataReader(f'{ticker}.SA', 'yahoo', '01-01-2015', '12-31-2021')
            ticker_df.index.name = 'DT_FIM_EXERC'
            ticker_df.index = pd.PeriodIndex(ticker_df.index, freq='Q')
            quantile_80 = ticker_df.resample('Q')['Close'].quantile(0.8).reset_index()
            quantile_80['Ticker']=ticker
            quantile_80 = quantile_80.groupby(['Ticker', 'DT_FIM_EXERC'])['Close'].mean()
            #display(quantile_80)
            prices = prices.append(quantile_80)
            prices_shift = prices_shift.append(quantile_80.shift(-1))
            print(prices, prices_shift)
        # If no information is retrieved, them all the rows regarding the ticker in analysis are removed.
        except:
            print(0)
prices.name = 'Current Quarter 80st pct'
prices_shift.name='Following Quarter 80st pct'
df.join(prices).join(prices_shift).loc['ELET3']

Ticker  DT_FIM_EXERC
NaN     NaN                   NaN
ELET3   2015Q1           5.577880
        2015Q2           7.297227
        2015Q3           5.795797
        2015Q4           5.987724
        2016Q1           6.795417
        2016Q2           8.862632
        2016Q3          20.694136
        2016Q4          23.690996
        2017Q1          22.327516
        2017Q2          16.501730
        2017Q3          20.006397
        2017Q4          21.227933
        2018Q1          23.641016
        2018Q2          19.042764
        2018Q3          16.591695
        2018Q4          24.710609
        2019Q1          36.796017
        2019Q2          35.276595
        2019Q3          44.575062
        2019Q4          38.205482
        2020Q1          39.470000
        2020Q2          29.940001
        2020Q3          37.676000
        2020Q4          35.972001
        2021Q1          33.812000
        2021Q2          44.996000
        2021Q3          42.792001
        2021Q4          36.

,1,1.01,1.01.01,1.01.02,1.01.02.01,1.01.02.01.01,1.01.02.01.02,1.01.02.02,1.01.02.02.01,1.01.02.03,1.01.03,1.01.03.01,1.01.03.02,1.01.04,1.01.05,1.01.06,1.01.06.01,1.01.07,1.01.08,1.01.08.01,1.01.08.02,1.01.08.03,1.01.09,1.01.09.03,1.02,1.02.01,1.02.01.01,1.02.01.01.01,1.02.01.01.02,1.02.01.02,1.02.01.02.01,1.02.01.03,1.02.01.03.01,1.02.01.03.02,1.02.01.04,1.02.01.05,1.02.01.06,1.02.01.06.01,1.02.01.07,1.02.01.08,1.02.01.08.01,1.02.01.08.03,1.02.01.08.04,1.02.01.09,1.02.01.09.01,1.02.01.09.02,1.02.01.10,1.02.01.10.01,1.02.01.10.02,1.02.01.11,1.02.01.11.01,1.02.02,1.02.02.01,1.02.02.01.01,1.02.02.01.04,1.02.02.01.05,1.02.02.01.07,1.02.02.02,1.02.03,1.02.03.01,1.02.03.02,1.02.03.03,1.02.04,1.02.04.01,1.02.04.01.01,1.02.04.02,1.02.04.03,1.02.04.04,1.02.04.05,1.02.04.08,1.03,1.03.01,1.03.02,1.04,1.04.01,1.04.02,1.04.03,1.05,1.05.01,1.05.02,1.05.03,1.05.04,1.06,1.06.01,1.06.02,1.06.03,1.07,1.07.01,1.07.02,1.07.03,1.08,1.08.01,1.08.02,2,2.01,2.01.01,2.01.01.01,2.01.01.02,2.01.02,2.01.02.01,2.01.02.02,2.01.03,2.01.03.01,2.01.03.01.01,2.01.03.02,2.01.03.03,2.01.04,2.01.04.01,2.01.04.01.01,2.01.04.01.02,2.01.04.02,2.01.04.03,2.01.05,2.01.05.01,2.01.05.01.01,2.01.05.01.03,2.01.05.01.04,2.01.05.02,2.01.05.02.01,2.01.05.02.02,2.01.05.02.03,2.01.06,2.01.06.01,2.01.06.01.01,2.01.06.01.02,2.01.06.01.03,2.01.06.01.04,2.01.06.02,2.01.06.02.01,2.01.06.02.02,2.01.06.02.03,2.01.07,2.01.07.01,2.01.07.02,2.02,2.02.01,2.02.01.01,2.02.01.01.01,2.02.01.01.02,2.02.01.02,2.02.01.03,2.02.01.03.01,2.02.01.04,2.02.01.05,2.02.02,2.02.02.01,2.02.02.01.01,2.02.02.01.03,2.02.02.01.04,2.02.02.02,2.02.02.02.01,2.02.02.02.02,2.02.03,2.02.03.01,2.02.04,2.02.04.01,2.02.04.01.01,2.02.04.01.02,2.02.04.01.03,2.02.04.01.04,2.02.04.02,2.02.04.02.01,2.02.04.02.02,2.02.04.02.03,2.02.05,2.02.05.01,2.02.05.02,2.02.06,2.02.06.01,2.02.06.02,2.02.06.03,2.03,2.03.01,2.03.02,2.03.02.01,2.03.02.02,2.03.02.03,2.03.02.04,2.03.02.05,2.03.02.06,2.03.03,2.03.03.01,2.03.03.02,2.03.04,2.03.04.01,2.03.04.02,2.03.04.03,2.03.04.04,2.03.04.05,2.03.04.06,2.03.04.07,2.03.04.08,2.03.04.09,2.03.05,2.03.06,2.03.07,2.03.08,2.03.09,2.04,2.05,2.06,2.06.01,2.07,2.07.01,2.07.01.01,2.07.01.02,2.07.01.02.01,2.07.01.04,2.07.01.04.01,2.07.01.04.04,2.07.01.05,2.07.01.06,2.07.01.08,2.07.02,2.08,2.08.01,2.08.02,2.08.02.01,2.08.02.05,2.08.03,2.08.04,2.08.04.01,2.08.04.02,2.08.04.04,2.08.04.07,2.08.04.09,2.08.05,2.08.06,2.08.07,2.08.08,2.08.09,6.01,6.02,6.03,6.04,6.05,6.05.01,6.05.02,5.01,5.02,5.03,5.04,5.04.01,5.04.02,5.04.03,5.04.04,5.04.05,5.04.06,5.04.07,5.05,5.05.01,5.05.02,5.05.02.01,5.05.02.02,5.05.02.03,5.05.02.04,5.05.02.05,5.05.03,5.05.03.01,5.06,5.06.01,5.06.02,5.06.03,5.07,4.01,4.02,4.02.01,4.02.02,4.03,4.03.01,4.03.02,4.04,4.04.01,4.04.02,3.01,3.01.01,3.01.01.01,3.01.01.02,3.01.02,3.01.02.01,3.01.03,3.01.03.02,3.01.04,3.01.05,3.01.06,3.02,3.02.01,3.02.01.01,3.02.01.02,3.02.01.03,3.02.02,3.02.02.01,3.02.02.02,3.02.02.03,3.03,3.04,3.04.01,3.04.02,3.04.03,3.04.03.01,3.04.03.02,3.04.03.03,3.04.03.04,3.04.04,3.04.05,3.04.05.01,3.04.05.02,3.04.05.03,3.04.05.04,3.04.05.05,3.04.05.06,3.04.06,3.04.06.01,3.04.06.02,3.04.06.03,3.04.06.04,3.04.06.05,3.04.06.06,3.04.07,3.04.08,3.05,3.06,3.06.01,3.06.02,3.07,3.08,3.08.01,3.08.02,3.09,3.09.01,3.09.02,3.10,3.10.01,3.10.02,3.11,3.11.01,3.11.02,3.13,3.13.01,3.13.02,7.01,7.01.01,7.01.02,7.01.03,7.01.04,7.01.05,7.02,7.02.01,7.02.02,7.02.03,7.02.04,7.03,7.03.01,7.03.02,7.03.03,7.03.04,7.04,7.04.01,7.04.02,7.04.03,7.04.05,7.05,7.05.01,7.05.02,7.05.05,7.06,7.06.01,7.06.02,7.06.03,7.07,7.07.01,7.08,7.08.01,7.08.01.01,7.08.01.02,7.08.01.03,7.08.01.04,7.08.02,7.08.02.01,7.08.02.02,7.08.02.03,7.08.03,7.08.03.01,7.08.03.02,7.08.03.03,7.08.04,7.08.04.01,7.08.04.02,7.08.04.03,7.08.04.04,7.08.05,7.09,7.09.01,7.09.01.01,7.09.01.02,7.09.01.03,7.09.01.04,7.09.02,7.09.02.01,7.09.02.02,7.09.02.03,7.09.03,7.09.03.01,7.09.04,7.09.04.01,7.09.04.02,7.09.04.03,7.09.04.04,7.09.05,7.10,7.11,7.11.01,7.11.01.01,7.11.01.02,7.11.01.03,7.11.01.04,7.11.02,7.11.02.01,7.11.02.02,7.11.02.03,7.11.0

In [701]:
x= pd.read_csv('Statements/itr_cia_aberta_BPA_con_2019.csv', sep=';', encoding='latin1')

In [706]:
x[(x.CNPJ_CIA=='00.001.180/0001-26') & (x.DT_FIM_EXERC=='2016-12-31')].ORDEM_EXERC.unique()#.DT_FIM_EXERC.unique()
x[(x.CNPJ_CIA=='00.001.180/0001-26')& (x.DT_FIM_EXERC=='2019-09-30')]#.ORDEM_EXERC.unique()

,CNPJ_CIA,DT_REFER,VERSAO,DENOM_CIA,CD_CVM,GRUPO_DFP,MOEDA,ESCALA_MOEDA,ORDEM_EXERC,DT_FIM_EXERC,CD_CONTA,DS_CONTA,VL_CONTA,ST_CONTA_FIXA
357,00.001.180/0001-26,2019-09-30,2,CENTRAIS ELET BRAS S.A. - ELETROBRAS,2437,DF Consolidado - Balanço Patrimonial Ativo,REAL,MIL,ÚLTIMO,2019-09-30,1,Ativo Total,177207066.0,S
359,00.001.180/0001-26,2019-09-30,2,CENTRAIS ELET BRAS S.A. - ELETROBRAS,2437,DF Consolidado - Balanço Patrimonial Ativo,REAL,MIL,ÚLTIMO,2019-09-30,1.01,Ativo Circulante,38819290.0,S
361,00.001.180/0001-26,2019-09-30,2,CENTRAIS ELET BRAS S.A. - ELETROBRAS,2437,DF Consolidado - Balanço Patrimonial Ativo,REAL,MIL,ÚLTIMO,2019-09-30,1.01.01,Caixa e Equivalentes de Caixa,3523628.0,S
363,00.001.180/0001-26,2019-09-30,2,CENTRAIS ELET BRAS S.A. - ELETROBRAS,2437,DF Consolidado - Balanço Patrimonial Ativo,REAL,MIL,ÚLTIMO,2019-09-30,1.01.01.01,Caixa,447738.0,N
365,00.001.180/0001-26,2019-09-30,2,CENTRAIS ELET BRAS S.A. - ELETROBRAS,2437,DF Consolidado - Balanço Patrimonial Ativo,REAL,MIL,ÚLTIMO,2019-09-30,1.01.01.02,Caixa Restrito,3075890.0,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
523,00.001.180/0001-26,2019-09-30,2,CENTRAIS ELET BRAS S.A. - ELETROBRAS,2437,DF Consolidado - Balanço Patrimonial Ativo,REAL,MIL,ÚLTIMO,2019-09-30,1.02.04,Intangível,633198.0,S
525,00.001.180/0001-26,2019-09-30,2,CENTRAIS ELET BRAS S.A. - ELETROBRAS,2437,DF Consolidado - Balanço Patrimonial Ativo,REAL,MIL,ÚLTIMO,2019-09-30,1.02.04.01,Intangíveis,633198.0,S
527,00.001.180/0001-26,2019-09-30,2,CENTRAIS ELET BRAS S.A. - ELETROBRAS,2437,DF Consolidado - Balanço Patrimonial Ativo,REAL,MIL,ÚLTIMO,2019-09-30,1.02.04.01.01,Contrato de Concessão,0.0,S
529,00.001.180/0001-26,2019-09-30,2,CENTRAIS ELET BRAS S.A. - ELETROBRAS,2437,DF Consolidado - Balanço Patrimonial Ativo,REAL,MIL,ÚLTIMO,2019-09-30,1.02.04.01.02,Outros,633198.0,N


In [346]:
for ticker in ['PETR3', 'CIEL3']:
    try:
        ticker_df = web.DataReader(f'{ticker}.SA', 'yahoo', '01-01-2015', '31-12-2021')
        #current_price = ticker_df['Close']
        quantile_80 = ticker_df.resample('Q')['Close'].quantile(0.8)
        volume = ticker_df.resample('Q')['Volume'].mean()
        display(volume)#quantile_80.shift(-1), quantile_80)
    except:
        print(ticker)
    #x.head()

Date
2015-03-31    1.979774e+07
2015-06-30    1.956939e+07
2015-09-30    1.826098e+07
2015-12-31    1.495182e+07
2016-03-31    1.840932e+07
2016-06-30    9.865154e+06
2016-09-30    9.721145e+06
2016-12-31    1.124784e+07
2017-03-31    8.508040e+06
2017-06-30    9.098037e+06
2017-09-30    7.286505e+06
2017-12-31    7.219391e+06
2018-03-31    9.591625e+06
2018-06-30    1.414961e+07
2018-09-30    8.851443e+06
2018-12-31    1.298199e+07
2019-03-31    1.009382e+07
2019-06-30    1.116817e+07
2019-09-30    9.401140e+06
2019-12-31    9.884677e+06
2020-03-31    3.233468e+07
2020-06-30    2.339408e+07
2020-09-30    1.655223e+07
2020-12-31    1.834159e+07
2021-03-31    2.878404e+07
2021-06-30    2.130579e+07
2021-09-30    1.628185e+07
2021-12-31    2.049832e+07
Freq: Q-DEC, Name: Volume, dtype: float64

Date
2015-03-31    6.967746e+06
2015-06-30    5.657107e+06
2015-09-30    6.717634e+06
2015-12-31    6.462331e+06
2016-03-31    9.333840e+06
2016-06-30    4.727733e+06
2016-09-30    4.929548e+06
2016-12-31    6.929410e+06
2017-03-31    8.209628e+06
2017-06-30    6.897090e+06
2017-09-30    6.191812e+06
2017-12-31    5.234132e+06
2018-03-31    6.283970e+06
2018-06-30    7.470708e+06
2018-09-30    6.728543e+06
2018-12-31    1.348679e+07
2019-03-31    1.607563e+07
2019-06-30    1.581259e+07
2019-09-30    1.916107e+07
2019-12-31    1.498990e+07
2020-03-31    1.786775e+07
2020-06-30    3.780142e+07
2020-09-30    5.659703e+07
2020-12-31    4.060255e+07
2021-03-31    4.191054e+07
2021-06-30    2.734439e+07
2021-09-30    2.808938e+07
2021-12-31    3.323636e+07
Freq: Q-DEC, Name: Volume, dtype: float64

In [342]:
x.drop(['PETR3'],level=0)

1        1.01    1.01.01    1.01.02  \
Ticker DT_FIM_EXERC                                                  
ELET3  2015Q1        152427122.0  34450167.0  2879958.0  4648826.0   
       2015Q2        150896328.0  31126437.0  2152413.0  7011131.0   
       2015Q3        152671980.0  30441004.0  2962477.0  6922589.0   
       2016Q1        149025438.0  28637497.0  3176163.0  5438598.0   
       2016Q2        173048005.0  27273549.0  2716600.0  6239175.0   
...                          ...         ...        ...        ...   
DXCO3  2020Q2         10969846.0   3960344.0  1598224.0        NaN   
       2020Q3         11208491.0   3958054.0  1559745.0        NaN   
       2021Q1         11223423.0   3979630.0  1262001.0        NaN   
       2021Q2         11890124.0   4219333.0  1326340.0        NaN   
       2021Q3         12470872.0   4571755.0  1406024.0        NaN   

                     1.01.02.01  1.01.02.01.01  1.01.02.01.02  1.01.02.02  \
Ticker DT_FIM_EXERC                                                         
ELET3  2015Q1               NaN            NaN            NaN   4648826.0   
       2015Q2               NaN            NaN            NaN   7011131.0   
       2015Q3               NaN            NaN            NaN   6922589.0   
       2016Q1               NaN            NaN            NaN   5438598.0   
       2016Q2               NaN            NaN            NaN   6239175.0   
...                         ...            ...            ...         ...   
DXCO3  2020Q2               NaN            NaN            NaN         NaN   
       2020Q3               NaN            NaN            NaN         NaN   
       2021Q1               NaN            NaN            NaN         NaN   
       2021Q2               NaN            NaN            NaN         NaN   
       2021Q3               NaN            NaN            NaN         NaN   

                     1.01.02.02.01  1.01.02.03    1.01.03  1.01.03.01  \
Ticker DT_FIM_EXERC                                                     
ELET3  2015Q1            4648826.0         NaN  4745292.0   4745292.0   
       2015Q2            7011131.0         NaN  4527083.0   4527083.0   
       2015Q3            6922589.0         NaN  5127948.0   5127948.0   
       2016Q1            5438598.0         NaN  4153132.0   4153132.0   
       2016Q2            6239175.0         NaN  4443974.0   4443974.0   
...                            ...         ...        ...         ...   
DXCO3  2020Q2                  NaN         NaN  1130705.0   1093233.0   
       2020Q3                  NaN         NaN  1316266.0   1280092.0   
       2021Q1                  NaN         NaN  1314680.0   1245734.0   
       2021Q2                  NaN         NaN  1429536.0   1354944.0   
       2021Q3                  NaN         NaN  1588502.0   1525885.0   

                     1.01.03.02    1.01.04  1.01.05   1.01.06  1.01.06.01  \
Ticker DT_FIM_EXERC                                                         
ELET3  2015Q1               NaN   873662.0      NaN  537106.0    537106.0   
       2015Q2               NaN   970021.0      NaN  666934.0    666934.0   
       2015Q3               NaN   947745.0      NaN  617931.0    617931.0   
       2016Q1               NaN   992093.0      NaN  499605.0    499605.0   
       2016Q2               NaN   996749.0      NaN  643438.0    643438.0   
...                         ...        ...      ...       ...         ...   
DXCO3  2020Q2           37472.0   962707.0      NaN  200925.0    200925.0   
       2020Q3           36174.0   892579.0      NaN  122983.0    122983.0   
       2021Q1           68946.0  1118733.0      NaN  151919.0    151919.0   
       2021Q2           74592.0  1167105.0      NaN  166868.0    166868.0   
       2021Q3           62617.0  1251370.0      NaN  196938.0    196938.0   

                     1.01.07     1.01.08  1.01.08.01  1.01.08.02  1.01.08.03  \
Ticker DT_FIM_EXERC                                                            
ELET3 

In [335]:
import pandas_datareader as web
for ticker in x.index.levels[0]:
    try:
        web.DataReader(f'{ticker}.SA', 'yahoo')
    except:
        print(ticker)

BBRK3
CESP3
GNDI3
IGTA3
LAME3
MOSI3
OMGE3
POWE3


In [332]:
np.array(x.index.levels[0])

array(['AALR3', 'AERI3', 'AESB3', 'AGRO3', 'AGXY3', 'ALLD3', 'ALPA3',
       'ALPK3', 'ALSO3', 'ALUP3', 'AMAR3', 'AMBP3', 'AMER3', 'ANIM3',
       'APER3', 'ARML3', 'ARZZ3', 'ASAI3', 'ATMP3', 'AVLL3', 'AZUL4',
       'B3SA3', 'BBRK3', 'BBSE3', 'BKBR3', 'BLAU3', 'BMOB3', 'BOAS3',
       'BRAP3', 'BRBI11', 'BRFS3', 'BRIT3', 'BRKM3', 'BRML3', 'BRPR3',
       'CAML3', 'CASH3', 'CCRO3', 'CEAB3', 'CEDO3', 'CESP3', 'CIEL3',
       'CLSA3', 'CLSC3', 'CMIG3', 'CMIN3', 'COGN3', 'CPFE3', 'CPLE3',
       'CRFB3', 'CSAN3', 'CSED3', 'CSMG3', 'CURY3', 'CVCB3', 'CXSE3',
       'CYRE3', 'DASA3', 'DESK3', 'DEXP3', 'DIRR3', 'DMVF3', 'DOTZ3',
       'DXCO3', 'ECOR3', 'EEEL3', 'EGIE3', 'ELET3', 'ELMD3', 'EMBR3',
       'ENAT3', 'ENBR3', 'ENEV3', 'ENGI3', 'ENJU3', 'EQTL3', 'ESPA3',
       'ETER3', 'EUCA3', 'EVEN3', 'EZTC3', 'FIQE3', 'FLRY3', 'FRAS3',
       'FRIO3', 'FRTA3', 'GFSA3', 'GGBR3', 'GGPS3', 'GMAT3', 'GNDI3',
       'GOAU3', 'GOLL4', 'GRND3', 'HAPV3', 'HBOR3', 'HBRE3', 'HBSA3',
       'HYPE3', 'IG

In [191]:
# 'acoes' lê a página que possui os CNPJ's e os Tickers das empresas.
acoes = pd.read_html('https://www.idinheiro.com.br/cnpj-empresas-listadas-b3/')[0].iloc[:, [1,2]]

<h3 style='font-size:30px;font-style:italic'> itr_cia_aberta</h3>

In [86]:
pd.read_csv('Statements/itr_cia_aberta_BPA_ind_2015.csv', encoding='ISO-8859-1', sep=';')#['DS_CONTA'].head(3)

FileNotFoundError: [Errno 2] No such file or directory: 'Statements/itr_cia_aberta_BPA_ind_2015.csv'

In [28]:
pd.read_csv('Statements/itr_cia_aberta_BPA_ind_2012.csv', encoding='ISO-8859-1', sep=';').head(3)

,CNPJ_CIA,DT_REFER,VERSAO,DENOM_CIA,CD_CVM,GRUPO_DFP,MOEDA,ESCALA_MOEDA,ORDEM_EXERC,DT_FIM_EXERC,CD_CONTA,DS_CONTA,VL_CONTA,ST_CONTA_FIXA
0,00.000.000/0001-91,2012-03-31,1,BCO BRASIL S.A.,1023,DF Individual - Balanço Patrimonial Ativo,REAL,MIL,PENÚLTIMO,2011-12-31,1,Ativo Total,890352257.0,S
1,00.000.000/0001-91,2012-03-31,1,BCO BRASIL S.A.,1023,DF Individual - Balanço Patrimonial Ativo,REAL,MIL,ÚLTIMO,2012-03-31,1,Ativo Total,913866693.0,S
2,00.000.000/0001-91,2012-03-31,1,BCO BRASIL S.A.,1023,DF Individual - Balanço Patrimonial Ativo,REAL,MIL,PENÚLTIMO,2011-12-31,1.01,Ativo Circulante,518716710.0,S


<h3 style='font-size:30px;font-style:italic'> BPA</h3>

In [83]:
x = pd.read_csv('Statements/itr_cia_aberta_BPA_con_2015.csv', encoding='ISO-8859-1', sep=';',parse_dates=['DT_FIM_EXERC'])
x[(x['CD_CONTA']=='1.02.01.10.02') & (x['ORDEM_EXERC']=='ÚLTIMO') & (x['CNPJ_CIA']=='00.001.180/0001-26')]

,CNPJ_CIA,DT_REFER,VERSAO,DENOM_CIA,CD_CVM,GRUPO_DFP,MOEDA,ESCALA_MOEDA,ORDEM_EXERC,DT_FIM_EXERC,CD_CONTA,DS_CONTA,VL_CONTA,ST_CONTA_FIXA
1,00.001.180/0001-26,2015-03-31,1,CENTRAIS ELET BRAS S.A. - ELETROBRAS,2437,DF Consolidado - Balanço Patrimonial Ativo,REAL,MIL,ÚLTIMO,2015-03-31,1,Ativo Total,152427122.0,S
167,00.001.180/0001-26,2015-06-30,1,CENTRAIS ELET BRAS S.A. - ELETROBRAS,2437,DF Consolidado - Balanço Patrimonial Ativo,REAL,MIL,ÚLTIMO,2015-06-30,1,Ativo Total,150896328.0,S
333,00.001.180/0001-26,2015-09-30,1,CENTRAIS ELET BRAS S.A. - ELETROBRAS,2437,DF Consolidado - Balanço Patrimonial Ativo,REAL,MIL,ÚLTIMO,2015-09-30,1,Ativo Total,152671980.0,S


<h3 style='font-size:30px;font-style:italic'> BPP</h3>

In [108]:
len('2.01.02.02')

10

In [109]:
import re
pd.pivot_table(x, values='VL_CONTA', columns='CD_CONTA', index=['CNPJ_CIA', 'DT_FIM_EXERC'])[[column for column in x['CD_CONTA'].unique() if len(column) in range(1,11)]]

CD_CONTA                                    2        2.01   2.01.01  \
CNPJ_CIA           DT_FIM_EXERC                                       
00.001.180/0001-26 2014-12-31    1.446317e+08  19284008.0       0.0   
                   2015-03-31    1.524271e+08  22083808.0       0.0   
                   2015-06-30    1.508963e+08  22287629.0       0.0   
                   2015-09-30    1.526720e+08  22549470.0       0.0   
00.070.698/0001-11 2014-12-31    2.750529e+06   1003217.0   36862.0   
...                                       ...         ...       ...   
92.929.520/0001-00 2015-09-30    1.001040e+05      9667.0       0.0   
97.837.181/0001-47 2014-12-31    8.797107e+06   1560728.0  149659.0   
                   2015-03-31    8.948191e+06   1329873.0  137016.0   
                   2015-06-30    8.859684e+06   1325319.0  147003.0   
                   2015-09-30    9.232477e+06   1158048.0  163446.0   

CD_CONTA                         2.01.01.01  2.01.01.02    2.01.02  \
CNPJ_CIA           DT_FIM_EXERC                                      
00.001.180/0001-26 2014-12-31           0.0         0.0  7489134.0   
                   2015-03-31           0.0         0.0  8701456.0   
                   2015-06-30           0.0         0.0  9193038.0   
                   2015-09-30           0.0         0.0  9261489.0   
00.070.698/0001-11 2014-12-31           0.0     36862.0   381304.0   
...                                     ...         ...        ...   
92.929.520/0001-00 2015-09-30           0.0         0.0     3103.0   
97.837.181/0001-47 2014-12-31           0.0         0.0   166832.0   
                   2015-03-31           0.0         0.0   193473.0   
                   2015-06-30           0.0         0.0   150712.0   
                   2015-09-30           0.0         0.0   169716.0   

CD_CONTA                         2.01.02.01  2.01.02.02    2.01.03  \
CNPJ_CIA           DT_FIM_EXERC                                      
00.001.180/0001-26 2014-12-31     7470482.0     18652.0  1186306.0   
                   2015-03-31     8671205.0     30251.0  1376171.0   
                   2015-06-30     9153694.0     39344.0  1617209.0   
                   2015-09-30     9250507.0     10982.0  1920551.0   
00.070.698/0001-11 2014-12-31      381304.0         0.0   184726.0   
...                                     ...         ...        ...   
92.929.520/0001-00 2015-09-30        3103.0         0.0     1616.0   
97.837.181/0001-47 2014-12-31           0.0         0.0    57758.0   
                   2015-03-31           0.0         0.0    82001.0   
                   2015-06-30           0.0         0.0    56332.0   
                   2015-09-30           0.0         0.0    67471.0   

CD_CONTA                         2.01.03.01  2.01.03.02  2.01.03.03  \
CNPJ_CIA           DT_FIM_EXERC                                       
00.001.180/0001-26 2014-12-31     1186306.0         0.0         0.0   
                   2015-03-31     1376171.0         0.0         0.0   
                   2015-06-30     1617209.0         0.0         0.0   
                   2015-09-30     1920551.0         0.0         0.0   
00.070.698/0001-11 2014-12-31       34441.0    148637.0      1648.0   
...                                     ...         ...         ...   
92.929.520/0001-00 2015-09-30        1616.0         0.0         0.0   
97.837.181/0001-47 2014-12-31           0.0         0.0         0.0   
                   2015-03-31           0.0         0.0         0.0   
                   2015-06-30           0.0         0.0         0.0   
                   2015-09-30           0.0         0.0         0.0   

CD_CONTA                           2.01.04  2.01.04.01  2.01.04.02  \
CNPJ_CIA           DT_FIM_EXERC                                      
00.001.180/0001-26 2014-12-31    5331770.0   4931531.0    325732.0   
                   2015-03-31    5869274.0   5389098.0    351446.0   
                   2015-06-30    5996980.0   55076

In [87]:
x = pd.read_csv('Statements/itr_cia_aberta_BPP_con_2015.csv', encoding='ISO-8859-1', sep=';')
x

,CNPJ_CIA,DT_REFER,VERSAO,DENOM_CIA,CD_CVM,GRUPO_DFP,MOEDA,ESCALA_MOEDA,ORDEM_EXERC,DT_FIM_EXERC,CD_CONTA,DS_CONTA,VL_CONTA,ST_CONTA_FIXA
0,00.001.180/0001-26,2015-03-31,1,CENTRAIS ELET BRAS S.A. - ELETROBRAS,2437,DF Consolidado - Balanço Patrimonial Passivo,REAL,MIL,PENÚLTIMO,2014-12-31,2,Passivo Total,144631697.0,S
1,00.001.180/0001-26,2015-03-31,1,CENTRAIS ELET BRAS S.A. - ELETROBRAS,2437,DF Consolidado - Balanço Patrimonial Passivo,REAL,MIL,ÚLTIMO,2015-03-31,2,Passivo Total,152427122.0,S
2,00.001.180/0001-26,2015-03-31,1,CENTRAIS ELET BRAS S.A. - ELETROBRAS,2437,DF Consolidado - Balanço Patrimonial Passivo,REAL,MIL,PENÚLTIMO,2014-12-31,2.01,Passivo Circulante,19284008.0,S
3,00.001.180/0001-26,2015-03-31,1,CENTRAIS ELET BRAS S.A. - ELETROBRAS,2437,DF Consolidado - Balanço Patrimonial Passivo,REAL,MIL,ÚLTIMO,2015-03-31,2.01,Passivo Circulante,22083808.0,S
4,00.001.180/0001-26,2015-03-31,1,CENTRAIS ELET BRAS S.A. - ELETROBRAS,2437,DF Consolidado - Balanço Patrimonial Passivo,REAL,MIL,PENÚLTIMO,2014-12-31,2.01.01,Obrigações Sociais e Trabalhistas,0.0,S
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
216667,97.837.181/0001-47,2015-09-30,1,DURATEX S.A.,21091,DF Consolidado - Balanço Patrimonial Passivo,REAL,MIL,ÚLTIMO,2015-09-30,2.03.07,Ajustes Acumulados de Conversão,0.0,S
216668,97.837.181/0001-47,2015-09-30,1,DURATEX S.A.,21091,DF Consolidado - Balanço Patrimonial Passivo,REAL,MIL,PENÚLTIMO,2014-12-31,2.03.08,Outros Resultados Abrangentes,0.0,S
216669,97.837.181/0001-47,2015-09-30,1,DURATEX S.A.,21091,DF Consolidado - Balanço Patrimonial Passivo,REAL,MIL,ÚLTIMO,2015-09-30,2.03.08,Outros Resultados Abrangentes,0.0,S
216670,97.837.181/0001-47,2015-09-30,1,DURATEX S.A.,21091,DF Consolidado - Balanço Patrimonial Passivo,REAL,MIL,PENÚLTIMO,2014-12-31,2.03.09,Participação dos Acionistas Não Controladores,65801.0,S


<h3 style='font-size:30px;font-style:italic'> DFC_MD</h3>

In [31]:
pd.read_csv('Statements/itr_cia_aberta_DFC_MD_con_2011.csv', encoding='ISO-8859-1', sep=';').head(3)

,CNPJ_CIA,DT_REFER,VERSAO,DENOM_CIA,CD_CVM,GRUPO_DFP,MOEDA,ESCALA_MOEDA,ORDEM_EXERC,DT_INI_EXERC,DT_FIM_EXERC,CD_CONTA,DS_CONTA,VL_CONTA,ST_CONTA_FIXA
0,00.070.698/0001-11,2011-03-31,1,CIA ENERGETICA DE BRASILIA,14451,DF Consolidado - Demonstração do Fluxo de Caix...,REAL,MIL,PENÚLTIMO,2010-01-01,2010-03-31,6.01,Caixa Líquido Atividades Operacionais,14679.0,S
1,00.070.698/0001-11,2011-03-31,1,CIA ENERGETICA DE BRASILIA,14451,DF Consolidado - Demonstração do Fluxo de Caix...,REAL,MIL,ÚLTIMO,2011-01-01,2011-03-31,6.01,Caixa Líquido Atividades Operacionais,18238.0,S
2,00.070.698/0001-11,2011-03-31,1,CIA ENERGETICA DE BRASILIA,14451,DF Consolidado - Demonstração do Fluxo de Caix...,REAL,MIL,PENÚLTIMO,2010-01-01,2010-03-31,6.01.01,Recebimento de Consumidores,472870.0,N


<h3 style='font-size:30px;font-style:italic'> DFC_MI</h3>

In [39]:
pd.read_csv('Statements/itr_cia_aberta_DFC_MI_con_2011.csv', encoding='ISO-8859-1', sep=';').head(3)

,CNPJ_CIA,DT_REFER,VERSAO,DENOM_CIA,CD_CVM,GRUPO_DFP,MOEDA,ESCALA_MOEDA,ORDEM_EXERC,DT_INI_EXERC,DT_FIM_EXERC,CD_CONTA,DS_CONTA,VL_CONTA,ST_CONTA_FIXA
0,00.001.180/0001-26,2011-03-31,1,CENTRAIS ELET BRAS S.A. - ELETROBRAS,2437,DF Consolidado - Demonstração do Fluxo de Caix...,REAL,MIL,PENÚLTIMO,2010-01-01,2010-03-31,6.01,Caixa Líquido Atividades Operacionais,1619666.0,S
1,00.001.180/0001-26,2011-03-31,1,CENTRAIS ELET BRAS S.A. - ELETROBRAS,2437,DF Consolidado - Demonstração do Fluxo de Caix...,REAL,MIL,ÚLTIMO,2011-01-01,2011-03-31,6.01,Caixa Líquido Atividades Operacionais,1189117.0,S
2,00.001.180/0001-26,2011-03-31,1,CENTRAIS ELET BRAS S.A. - ELETROBRAS,2437,DF Consolidado - Demonstração do Fluxo de Caix...,REAL,MIL,PENÚLTIMO,2010-01-01,2010-03-31,6.01.01,Caixa Gerado nas Operações,2407485.0,S


<h3 style='font-size:30px;font-style:italic'> DMPL</h3>

In [52]:
pd.read_csv('Statements/itr_cia_aberta_DMPL_con_2015.csv', encoding='ISO-8859-1', sep=';')['VL_CONTA'].max()

442198000.0

<h3 style='font-size:30px;font-style:italic'> DRA</h3>

In [41]:
pd.read_csv('Statements/itr_cia_aberta_DRA_con_2011.csv', encoding='ISO-8859-1', sep=';').head(3)

,CNPJ_CIA,DT_REFER,VERSAO,DENOM_CIA,CD_CVM,GRUPO_DFP,MOEDA,ESCALA_MOEDA,ORDEM_EXERC,DT_INI_EXERC,DT_FIM_EXERC,CD_CONTA,DS_CONTA,VL_CONTA,ST_CONTA_FIXA
0,00.001.180/0001-26,2011-03-31,1,CENTRAIS ELET BRAS S.A. - ELETROBRAS,2437,DF Consolidado - Demonstração de Resultado Abr...,REAL,MIL,PENÚLTIMO,2010-01-01,2010-03-31,4.01,Lucro Líquido Consolidado do Período,0.0,S
1,00.001.180/0001-26,2011-03-31,1,CENTRAIS ELET BRAS S.A. - ELETROBRAS,2437,DF Consolidado - Demonstração de Resultado Abr...,REAL,MIL,ÚLTIMO,2011-01-01,2011-03-31,4.01,Lucro Líquido Consolidado do Período,0.0,S
2,00.001.180/0001-26,2011-03-31,1,CENTRAIS ELET BRAS S.A. - ELETROBRAS,2437,DF Consolidado - Demonstração de Resultado Abr...,REAL,MIL,PENÚLTIMO,2010-01-01,2010-03-31,4.02,Outros Resultados Abrangentes,0.0,S


<h3 style='font-size:30px;font-style:italic'> DRE</h3>

In [42]:
pd.read_csv('Statements/itr_cia_aberta_DRE_con_2011.csv', encoding='ISO-8859-1', sep=';').head(3)

,CNPJ_CIA,DT_REFER,VERSAO,DENOM_CIA,CD_CVM,GRUPO_DFP,MOEDA,ESCALA_MOEDA,ORDEM_EXERC,DT_INI_EXERC,DT_FIM_EXERC,CD_CONTA,DS_CONTA,VL_CONTA,ST_CONTA_FIXA
0,00.001.180/0001-26,2011-03-31,1,CENTRAIS ELET BRAS S.A. - ELETROBRAS,2437,DF Consolidado - Demonstração do Resultado,REAL,MIL,PENÚLTIMO,2010-01-01,2010-03-31,3.01,Receita de Venda de Bens e/ou Serviços,6159172.0,S
1,00.001.180/0001-26,2011-03-31,1,CENTRAIS ELET BRAS S.A. - ELETROBRAS,2437,DF Consolidado - Demonstração do Resultado,REAL,MIL,ÚLTIMO,2011-01-01,2011-03-31,3.01,Receita de Venda de Bens e/ou Serviços,8551982.0,S
2,00.001.180/0001-26,2011-03-31,1,CENTRAIS ELET BRAS S.A. - ELETROBRAS,2437,DF Consolidado - Demonstração do Resultado,REAL,MIL,PENÚLTIMO,2010-01-01,2010-03-31,3.02,Custo dos Bens e/ou Serviços Vendidos,-960060.0,S


<h3 style='font-size:30px;font-style:italic'> DVA</h3>

In [79]:
#f = lambda x: x[x['ORDEM_EXERC']=='ÚLTIMO']
pd.read_csv('Statements/itr_cia_aberta_DVA_con_2011.csv', encoding='ISO-8859-1', sep=';')#.apply(f, axis=1)#.head(3)

,CNPJ_CIA,DT_REFER,VERSAO,DENOM_CIA,CD_CVM,GRUPO_DFP,MOEDA,ESCALA_MOEDA,ORDEM_EXERC,DT_INI_EXERC,DT_FIM_EXERC,CD_CONTA,DS_CONTA,VL_CONTA,ST_CONTA_FIXA
0,00.001.180/0001-26,2011-03-31,1,CENTRAIS ELET BRAS S.A. - ELETROBRAS,2437,DF Consolidado - Demonstração de Valor Adicionado,REAL,MIL,PENÚLTIMO,2010-01-01,2010-03-31,7.01,Receitas,6354944.0,S
1,00.001.180/0001-26,2011-03-31,1,CENTRAIS ELET BRAS S.A. - ELETROBRAS,2437,DF Consolidado - Demonstração de Valor Adicionado,REAL,MIL,ÚLTIMO,2011-01-01,2011-03-31,7.01,Receitas,8753827.0,S
2,00.001.180/0001-26,2011-03-31,1,CENTRAIS ELET BRAS S.A. - ELETROBRAS,2437,DF Consolidado - Demonstração de Valor Adicionado,REAL,MIL,PENÚLTIMO,2010-01-01,2010-03-31,7.01.01,"Vendas de Mercadorias, Produtos e Serviços",6156290.0,S
3,00.001.180/0001-26,2011-03-31,1,CENTRAIS ELET BRAS S.A. - ELETROBRAS,2437,DF Consolidado - Demonstração de Valor Adicionado,REAL,MIL,ÚLTIMO,2011-01-01,2011-03-31,7.01.01,"Vendas de Mercadorias, Produtos e Serviços",8551982.0,S
4,00.001.180/0001-26,2011-03-31,1,CENTRAIS ELET BRAS S.A. - ELETROBRAS,2437,DF Consolidado - Demonstração de Valor Adicionado,REAL,MIL,PENÚLTIMO,2010-01-01,2010-03-31,7.01.02,Outras Receitas,198654.0,S
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89505,97.837.181/0001-47,2011-09-30,1,DURATEX S.A.,21091,DF Consolidado - Demonstração de Valor Adicionado,REAL,MIL,ÚLTIMO,2011-01-01,2011-09-30,7.08.04.03,Lucros Retidos / Prejuízo do Período,187980.0,S
89506,97.837.181/0001-47,2011-09-30,1,DURATEX S.A.,21091,DF Consolidado - Demonstração de Valor Adicionado,REAL,MIL,PENÚLTIMO,2010-01-01,2010-09-30,7.08.04.04,Part. Não Controladores nos Lucros Retidos,283.0,S
89507,97.837.181/0001-47,2011-09-30,1,DURATEX S.A.,21091,DF Consolidado - Demonstração de Valor Adicionado,REAL,MIL,ÚLTIMO,2011-01-01,2011-09-30,7.08.04.04,Part. Não Controladores nos Lucros Retidos,525.0,S
89508,97.837.181/0001-47,2011-09-30,1,DURATEX S.A.,21091,DF Consolidado - Demonstração de Valor Adicionado,REAL,MIL,PENÚLTIMO,2010-01-01,2010-09-30,7.08.05,Outros,0.0,S
